<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Generating-Word-Embeddings---Lab" data-toc-modified-id="Generating-Word-Embeddings---Lab-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Generating Word Embeddings - Lab</a></span><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Objectives" data-toc-modified-id="Objectives-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Objectives</a></span></li><li><span><a href="#Getting-Started" data-toc-modified-id="Getting-Started-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Getting Started</a></span></li><li><span><a href="#Preparing-the-Data" data-toc-modified-id="Preparing-the-Data-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Preparing the Data</a></span></li><li><span><a href="#Training-the-Model" data-toc-modified-id="Training-the-Model-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Training the Model</a></span></li><li><span><a href="#Examining-Your-Word-Vectors" data-toc-modified-id="Examining-Your-Word-Vectors-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Examining Your Word Vectors</a></span></li><li><span><a href="#Summary" data-toc-modified-id="Summary-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Summary</a></span></li></ul></li></ul></div>

# Generating Word Embeddings - Lab

## Introduction

In this lab, you'll learn how to generate word embeddings by training a Word2Vec model, and then embedding layers into Deep Neural Networks for NLP!

## Objectives

You will be able to:

* Demonstrate a basic understanding of the architecture of the Word2Vec model
* Demonstrate an understanding of the various tunable parameters of Word2Vec such as vector size and window size

## Getting Started

In this lab, you'll start by creating your own word embeddings by making use of the Word2Vec Model. Then, you'll move onto building Neural Networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in your model. 

As you've seen, the easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which you can use to begin training immediately. For this lab, you'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.

Run the cell below to import everything you'll need for this lab. 

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

Now, import the data. The data stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a github repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json` function from pandas to read the dataset into a DataFrame. Be sure to include the parameter `lines=True` when reading in the dataset!

Once you've loaded in the data, inspect the head of the DataFrame to see what your data looks like. 

In [2]:
raw_df = pd.read_json('News_Category_Dataset_v2.json', lines=True)
raw_df.head()

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."


## Preparing the Data

Since you're working with text data, you need to do some basic preprocessing including tokenization. Notice from the data sample that two different columns contain text data--`headline` and `short_description`. The more text data your Word2Vec model has, the better it will perform. Therefore, you'll want to combine the two columns before tokenizing each comment and training your Word2Vec model. 

In the cell below:

* Create a column called `combined_text` that consists of the data from `df.headline` plus a space character (`' '`) plus the data from `df.short_description`.
* Use the `combined_text` column's `map()` function and pass in `word_tokenize`. Store the result returned in `data`.

In [3]:
df = raw_df
df['combined_text'] = df.headline + '' + df.short_description
df.head()

,authors,category,date,headline,link,short_description,combined_text
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,There Were 2 Mass Shootings In Texas Last Week...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,Will Smith Joins Diplo And Nicky Jam For The 2...
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,Hugh Grant Marries For The First Time At Age 5...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,Jim Carrey Blasts 'Castrato' Adam Schiff And D...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",Julianna Margulies Uses Donald Trump Poop Bags...


In [6]:
data = df['combined_text'].map(word_tokenize)

Inspect the first 5 items in `data` to see how everything looks. 

In [7]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

Notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Remember that for a Word2Vec model you can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If your window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example you saw).

Now that you've prepared the data, train your model and explore a bit!

## Training the Model

Start by instantiating a Word2Vec Model from gensim below. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1`.
    * The number of threads to use during training, `workers=4`

In [9]:
model = Word2Vec(data, size=100, window=5, min_count=1, workers=4)

Now, that you've instantiated Word2Vec model, train it on your text data. 

In the cell below:

* Call `model.train()` and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which you can find in `model.corpus_count`. 
    * The number of `epochs` you want to train for, which we'll set to `10`

In [10]:
model.train(data, total_examples=model.corpus_count, epochs=10)

(54332520, 65902920)

Great! you now have a fully trained model! The word vectors themselves are stored inside of a `Word2VecKeyedVectors` instance, which is stored inside of `model.wv`. To simplify this, restore this object inside of the variable `wv` to save yourself some keystrokes down the line. 

In [11]:
wv = model.wv

## Examining Your Word Vectors

Now that you have a trained Word2Vec model, go ahead and explore the relationships between some of the words in the corpus! 

One cool thing you can use Word2Vec for is to get the most similar words to a given word. You can do this passing in the word to `wv.most_similar()`. 

In the cell below, try getting the most similar word to `'Texas'`.

In [12]:
wv.most_similar('Texas')

[('Pennsylvania', 0.8242433667182922),
 ('Ohio', 0.8025435209274292),
 ('Arkansas', 0.800093412399292),
 ('Louisiana', 0.7909924387931824),
 ('Oklahoma', 0.7855998277664185),
 ('Oregon', 0.7844865322113037),
 ('Florida', 0.7709092497825623),
 ('Georgia', 0.765152096748352),
 ('Illinois', 0.7618453502655029),
 ('Michigan', 0.7601596117019653)]

Interesting! All of the most similar words are also states. 

You can also get the least similar vectors to a given word by passing in the word to the `most_similar()` function's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [13]:
wv.most_similar(negative='Texas')

[('ProNever', 0.49911898374557495),
 ('AskSo', 0.4799977242946625),
 ('Proposal5', 0.46394646167755127),
 ('YearSome', 0.46269601583480835),
 ('BlocFurious', 0.46006858348846436),
 ('Them.Santa', 0.45210710167884827),
 ("Fear'Do", 0.44629716873168945),
 ('NephewThe', 0.44545090198516846),
 ('AffirmationsWhen', 0.4440603256225586),
 ('SeeMany', 0.4436493217945099)]

This seems like random noise. It is a result of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with your given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

You can also get the vector for a given word by passing in the word as if you were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [14]:
wv['Texas']

array([-2.035695  , -0.46909514, -1.8768532 , -0.63773894,  0.15587516,
        0.56874067, -2.485352  , -1.2748755 , -0.99434197,  2.3241432 ,
       -0.277332  ,  1.0465543 ,  1.6752175 ,  0.74800384, -2.1039236 ,
       -1.8721114 , -1.40857   ,  0.4047052 ,  3.4677646 , -0.6690689 ,
       -0.66653985,  1.0512558 ,  0.6079532 , -0.17228761, -0.9435925 ,
        1.1683182 , -2.001674  , -1.0521787 ,  0.02908624,  1.3202245 ,
        0.06208661, -1.806479  , -0.3854091 ,  0.03800237, -0.11922952,
       -1.8906548 , -0.3167714 , -0.28822953, -0.78841954,  0.55057526,
       -0.34293455,  0.25607482, -1.372618  , -1.1577047 ,  2.511078  ,
        1.0839618 , -1.5789862 , -0.9071697 ,  1.0099245 , -0.10721759,
        0.8029698 ,  2.7361894 , -2.3607414 , -1.6138065 ,  0.19831891,
       -1.3231256 ,  0.33883807, -2.8986394 ,  0.23023331,  1.5230174 ,
        1.6186132 , -1.5515627 ,  0.18538317, -0.04632144, -3.430605  ,
       -1.0871694 ,  0.34615862, -0.23611508,  0.7039794 , -2.58

Now get all of the word vectors from the object at once. You can find these inside of `wv.vectors`. Try it out in the cell below.  

In [15]:
wv.vectors

array([[ 9.3285477e-01,  2.9958642e-01,  8.7037104e-01, ...,
        -6.8559337e-01,  5.9393358e-01, -1.8629158e+00],
       [ 3.5405974e+00,  5.1290363e-01,  2.0374811e+00, ...,
        -1.7056949e+00,  2.0115678e+00, -2.8865168e+00],
       [-3.2218596e-01,  1.1578466e+00,  3.2967796e+00, ...,
         6.2906688e-01, -2.4336505e+00, -1.5936890e+00],
       ...,
       [-3.8615227e-02,  1.7709566e-02, -4.5577958e-02, ...,
        -1.9575400e-02,  1.3311200e-01,  1.9866783e-02],
       [-3.8277183e-02, -1.7768435e-02, -1.8411957e-02, ...,
        -6.8832517e-02,  2.7259065e-02,  4.5075737e-02],
       [ 1.1086527e-02,  1.9309844e-03, -4.0549900e-02, ...,
         1.8747156e-02, -3.3215061e-02,  4.2869136e-02]], dtype=float32)

As a final exercise, try to recreate the _'king' - 'man' + 'woman' = 'queen'_ example previously mentioned. You can do this by using the `most_similar` function and translating the word analogies into an addition/subtraction formulation (as shown above). Pass the original comparison, which you are calculating a difference between, to the negative parameter, and the analogous starter you want to apply the same transformation to, to the `positive` parameter.

Do this now in the cell below. 

In [17]:
wv.most_similar(positive=['king', 'woman'], negative=['man'])

[('princess', 0.6699044108390808),
 ('title', 0.6438217163085938),
 ('revival', 0.6373555064201355),
 ('jewel', 0.628025233745575),
 ('villain', 0.6061543226242065),
 ('song', 0.6010643243789673),
 ('queen', 0.5980172157287598),
 ('gem', 0.5975761413574219),
 ('movie', 0.5975445508956909),
 ('coin', 0.5955368280410767)]

As you can see from the output above, your model isn't perfect, but 'Queen' is still in the top 3, and with 'Princess' not too far behind. As you can see from the word in first place, 'reminiscent', your model is far from perfect. This is likely because you didn't have enough training data. That said, given the small amount of training data provided, the model still performs remarkably well! 

In the next lab, you'll reinvestigate transfer learning, loading in the weights from an open-sourced model that has already been trained for a very long time on a massive amount of data. Specifically, you'll work with the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless your text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, you learned how to train and use a Word2Vec model to created vectorized word embeddings!